In [1]:
import pandas as pd 
import numpy as np

In [2]:
kc_dtypes = {'id': str, 'date' : str,  'price': float, 'bedrooms' : int, 'bathrooms' : float, 'sqft_living': int, 'sqft_lot': int, 'floors': float, 
             'waterfront': float, 'view' : float, 'condition': float, 'grade': int, 'sqft_above': int, 'yr_built': int,
             'yr_renovated': float, 'zipcode': float, 'lat': float, 'long': float, 'sqft_living15': int, 'sqft_lot15': int}

kc_data = pd.read_csv('dsc-phase-2-project/data/kc_house_data.csv', parse_dates = ['date'])

In [3]:
kc_parcels = pd.read_csv('dsc-phase-2-project/data/EXTR_Parcel.csv', encoding='ISO-8859-1')

i = 0
j = 0

kc_parcels['Major'] = kc_parcels['Major'].astype(str)
kc_parcels['Major'] = kc_parcels['Major'].str.strip()    
kc_parcels['Minor'] = kc_parcels['Minor'].astype(str)
kc_parcels['Minor'] = kc_parcels['Minor'].str.strip()

for row in kc_parcels:
    while len(str(kc_parcels['Minor'][i])) + j < 4:
             kc_parcels['Major'][i]*10
             j += 1
             
    i += 1
    
kc_parcels['id'] = kc_parcels['Major'] + kc_parcels['Minor']

In [4]:
kc_data.merge(kc_parcels, how='inner', left_on='id')

TypeError: object of type 'NoneType' has no len()

In [ ]:
kc_data.loc[kc_data['id'] ==  162405162405]

In [5]:
kc_parcels['id'] = kc_parcels[i]

,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,QuarterSection,...,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems,id
0,32305,9029,,NaN,,,5,23,3,,...,N,N,N,N,N,N,N,N,N,323059029
1,322207,9035,,NaN,,,7,22,32,,...,N,N,N,N,N,N,N,N,N,3222079035
2,52613,9002,,NaN,,,13,26,5,,...,N,N,N,N,N,N,N,N,N,526139002
3,766620,3435,,SEATTLE TIDE LDS BL 01-376,POR,248,4,24,8,NE,...,N,N,N,N,N,N,N,N,N,7666203435
4,508440,105,,MAPELS ADD,POR,3 &,4,24,20,NW,...,N,N,N,N,N,N,N,N,N,508440105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616243,881990,840,,UNIVERSITY HOME TRS,4 & 13,8,4,25,10,NW,...,N,N,N,N,N,N,N,N,N,881990840
616244,602150,990,,NELSON H E ADD,7,5,3,25,2,NE,...,N,N,N,N,N,N,N,N,N,602150990
616245,162405,9314,,NaN,,,5,24,16,NW,...,N,N,N,N,N,N,N,N,N,1624059314
616246,781280,885,,SKYWAY PARK ADD,11,9,4,23,12,SW,...,N,N,N,N,N,N,N,N,N,781280885
